
# Facebook

Collects information from Facebook using Selenium Webdriver.

## General Functions



### Login

Login to facebook by selecting and filling specific elements on the page.



#### the code

facebook login?

TODO: generalize this for multi-login code



In [ ]:

const getClient = importer.import('selenium client')
const { Builder, Browser, By, Key, until } = require('selenium-webdriver')

const PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
const PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'brian.json');

async function enterFacebook(driver) {
  console.log('Facebook: Sign in required');

  var credentials = require(PASSWORDS_FILE)

  //let body = await driver.findElement(By.css('body'))
  await driver.wait(until.elementLocated(By.css('.login-form, [type="submit"]')), 10000);
  let submit = await driver.findElement(By.css('.login-form, [type="submit"]'))

  let login = await driver.findElement(By.css('input[name*="email"]'))
  
  await driver.executeScript('arguments[0].click();', login)
  await driver.actions().sendKeys(credentials.username).perform()

  await new Promise(resolve => setTimeout(resolve, 1000))

  //await pass.sendKeys(credentials.username)
  //await driver.executeScript('arguments[0].value="' + credentials.username + '";', login)
  
  console.log('Facebook: Require password')

  let pass = await driver.findElement(By.css('input[name*="pass"]'))
  await pass.click()
  await pass.sendKeys(credentials.password)

  await submit.click()

  await new Promise(resolve => setTimeout(resolve, 3000))

  let loginStill, loginStill2
  try {
    loginStill = await driver.findElement(By.css('#captcha-internal'))
  } catch (e) {}
  try {
    loginStill2 = await driver.findElement(By.css('input[name*="session_password"]'))
  } catch (e) {}

  await new Promise(resolve => setTimeout(resolve, 4000))

  if(loginStill || loginStill2) {
    throw new Error('captcha')
  }
}

async function loginFacebook(driver) {
  if(!driver) {
    driver = await getClient();
  }

  try {
    let url = await driver.getCurrentUrl()
    let loggedIn = url.indexOf('facebook') > -1 && url.indexOf('login') == -1

    if(loggedIn) {
      if(await driver.findElement(By.xpath('iframe.authentication-iframe'))) {
        await driver.frame((await driver.element('iframe.authentication-iframe')).value)
        await enterFacebook()
        await frame()
      }
    } else {
      await driver.get('https://www.facebook.com/')
      let loginLink, loginLink2
      try {
        loginLink = await driver.findElement(By.xpath('//a[text()[contains(.,"Forgot password?")]]'))
      } catch (e) {}
      try {
        loginLink2 = await driver.findElement(By.xpath('//a[text()[contains(.,"Create new account")]]'))
      } catch (e) {}
      if(loginLink || loginLink2) {
        await enterFacebook(driver)
      }
    }
  } catch (e) {
    driver.quit() // avoid leaving sessions hanging around

    throw e
  }

  return driver
}

module.exports = loginFacebook



### List Friends



#### the code

list friends?


In [ ]:

//const fs = require('fs')
const getClient = importer.import('selenium client')
const getAllUntil = importer.import('all elements until')
const loginFacebook = importer.import('facebook login')
//const {FACEBOOK_PATH} = require('./config.js')
const selectDom = importer.import('selenium select')

async function listFriends(driver) {
  if (!driver) {
    driver = await getClient()
    await loginFacebook(driver)
  }

  try {
    let url = await driver.getCurrentUrl()
    let alreadyThere = url.indexOf('facebook') && url.indexOf('suggestions') > -1
    if (!alreadyThere) {
      await driver.get('https://www.facebook.com/friends/suggestions')
      await new Promise(resolve => setTimeout(resolve, 4000))
    }

    let result = await getAllUntil(selectDom, driver,
      '//div[@aria-label="Suggestions"]/div/div[2]',
      '//div[@aria-label="Suggestions"]//a[@role="link"]/@href',
      /* friends */[],
      (a, b) => a === b,
      (i) => i < 10
    )

    //fs.writeFileSync(
    //  FACEBOOK_PATH + '/friends.json',
    //  JSON.stringify(result, null, 4));

    return result.filter((l, i, arr) => arr.indexOf(l) === i)
  } catch (e) {
    //driver.quit() // avoid leaving sessions hanging around

    throw e
  }
}

module.exports = listFriends
